<a href="https://colab.research.google.com/github/MykolaWauer/Spiced_Academy_Team_Project/blob/foawzi-feature/supermarket_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install --upgrade opencv-python
#!pip install ofs

In [51]:
import numpy as np
import cv2

In [52]:
TILE_SIZE = 32
OFS = 50

In [53]:


MARKET = """
##################
##....c...c......#
##..DR..RS..SF..F#
##c.DRc.RS..SFc.F#
##..DR..RS.cSF..F#
##.cDR.cRS..SF..F#
##..DR..RS..SF..F#
##..c....c....c..#
##..CC..CC..CC...#
##.............c.#
##...............#
##WW##WW##WW##GG##
""".strip()



In [62]:
import numpy as np
import cv2

TILE_SIZE = 32
OFS = 50

class SupermarketMap:
    """Visualizes the supermarket background"""

    def __init__(self, layout, tiles):
        """
        layout : a string with each character representing a tile
        tile   : a numpy array containing the tile image
        """
        self.tiles = tiles
        self.contents = [list(row) for row in layout.split("\n")]
        self.xsize = len(self.contents[0])
        self.ysize = len(self.contents)
        self.image = np.zeros(
            (self.ysize * TILE_SIZE, self.xsize * TILE_SIZE, 3), dtype=np.uint8
        )
        self.prepare_map()

    def get_tile(self, char):
        """returns the array for a given tile character"""
        if char == "#":
          return self.tiles[0:32, 0:32]
        elif char == "G": # gates
            return self.tiles[8 * 32 : 9 * 32, 3 * 32 : 4 * 32] 
        elif char == "W": # window
            return self.tiles[8 * 32 : 9 * 32, 4 * 32 : 5 * 32]
        elif char == "C": # checko@          
          return self.tiles[2 * 32 : 3 * 32, 8 * 32 : 9 * 32]
        elif char == "F": # fruits
          return self.tiles[1 * 32 : 2 * 32, 4 * 32 : 5 * 32] 
        elif char == "S": # spices
            return self.tiles[1 * 32 : 2 * 32, 3 * 32 : 4 * 32]       
        elif char == "R": # dairy
            return self.tiles[8 * 32 : 9 * 32, 7 * 32 : 8 * 32] 
        elif char == "D": # drinks
            return self.tiles[6 * 32 : 7 * 32, 13 * 32 : 14 * 32] 
        elif char == "c": # customer/shopping cart
            return self.tiles[8 * 32 : 9 * 32, 6 * 32 : 7 * 32] 
        else:
            return self.tiles[32:64, 64:96]

    def prepare_map(self):
        """prepares the entire image as a big numpy array"""
        for y, row in enumerate(self.contents):
            for x, tile in enumerate(row):
                bm = self.get_tile(tile)
                self.image[
                    y * TILE_SIZE : (y + 1) * TILE_SIZE,
                    x * TILE_SIZE : (x + 1) * TILE_SIZE,
                ] = bm

    def draw(self, frame, offset=OFS):
        """
        draws the image into a frame
        offset pixels from the top left corner
        """
        frame[
            OFS : OFS + self.image.shape[0], OFS : OFS + self.image.shape[1]
        ] = self.image

    def write_image(self, filename):
        """writes the image into a file"""
        cv2.imwrite(filename, self.image)




In [63]:
import pandas as pd
import numpy as np
import cv2
import time
import os
os.path.exists('market.png') # Will likely give False
os.getcwd() # This will return the current working directory


matrix_monday = pd.read_csv('monday.csv', index_col = 0).T
matrix_monday = np.array(matrix_monday)
simulated_data = pd.read_csv('sim_c10_till_20.csv', sep=',')

TILE_SIZE = 32
OFS = 50

map = """
##################
#................#
#D..DR..RS..SF..F#
#D..DR..RS..SF..F#
#D..DR..RS..SF..F#
#D..DR..RS..SF..F#
#D..DR..RS..SF..F#
#................#
#...CC..CC..CC...#
#...CC..CC..CC...#
#................#
##WW##WW##WW##GG##
""".strip().split('\n')




MARKET = cv2.imread("market.png")  # load in the image of the supermarket as a NumPy array
TILES = cv2.imread("tiles.png")  # load in the tileset image as a NumPy array

class SupermarketMap:
    def __init__(self, market, tiles):
        self.map = market
        self.tiles = tiles

    def get_tile(self, char):
        if char == "#":
          return self.tiles[0:32, 0:32,:]
        elif char == "G": # gates
          return self.tiles[8 * 32 : 9 * 32, 3 * 32 : 4 * 32]
        elif char == "c": # customer
            return self.tiles[9 * 32 : 10 * 32, 0:32]
        elif char == "b": # basket
            return self.tiles[9 * 32 : 10 * 32, 7 * 32 : 8 * 32]

def add_customer(image, x, y):
    customer_image = SupermarketMap(MARKET, TILES).get_tile('c')
    image[y:y+32, x:x+32] = customer_image

def add_basket(image, x, y):
    basket_image = SupermarketMap(MARKET, TILES).get_tile('b')
    image[y:y+32, x:x+32] = basket_image

def add_gate(image, x, y):
    gate_image = SupermarketMap(MARKET, TILES).get_tile('G')
    image[y:y+32, x:x+32] = gate_image

def create_market_image():
    # create a black image with the same dimensions as the supermarket map
    market_image = np.zeros((MARKET.shape[0], MARKET.shape[1], 3), np.uint8)    
    # add the walls to the image
    wall_tile = SupermarketMap(MARKET, TILES).get_tile("#")
    for y in range(0, MARKET.shape[0], 32):
        for x in range(0, MARKET.shape[1], 32):
          if (MARKET[y][x] == [0, 0, 0]).all():
            market_image[y:y+32, x:x+32] = wall_tile
    
    # add gates, customers, and baskets to the image
        add_gate(market_image, 352, 0)
        add_gate(market_image, 416, 0)
        add_customer(market_image, 100, 100)
        add_basket(market_image, 200, 200)
    
        return market_image

# create the market image and display it
market_image = create_market_image()
cv2.imshow("Market", market_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: ignored

In [64]:
 
MARKET = """
##################
#................#
#D..DR..RS..SF..F#
#D..DR..RS..SF..F#
#D..DR..RS..SF..F#
#D..DR..RS..SF..F#
#D..DR..RS..SF..F#
#................#
#...CC..CC..CC...#
#...CC..CC..CC...#
#................#
##WW##WW##WW##GG##
""".strip().split('\n')

In [65]:
np.array(MARKET).shape

(12,)

In [66]:
MARKET

['##################',
 '#................#',
 '#D..DR..RS..SF..F#',
 '#D..DR..RS..SF..F#',
 '#D..DR..RS..SF..F#',
 '#D..DR..RS..SF..F#',
 '#D..DR..RS..SF..F#',
 '#................#',
 '#...CC..CC..CC...#',
 '#...CC..CC..CC...#',
 '#................#',
 '##WW##WW##WW##GG##']